# Imports

In [1]:
import pandas as pd

# Data Reading

In [14]:
dataset = pd.read_csv("IMDB Dataset.csv", sep=',')
# dataset = pd.read_csv("fruits.csv", sep=',')
display(dataset)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


# Data Split

In [18]:
# split data to positive and negative
p_samples = dataset[dataset['sentiment'] == 'positive']
n_samples = dataset[dataset['sentiment'] == 'negative']
# display(p_samples)
# display(n_samples)

# split the positive class samples to (70%, 10%, 20%) for (training, validation, testing) respectively 
p_testing = p_samples.sample(frac = 0.2)
p_validation = p_samples.drop(p_testing.index).sample(frac = 0.125)
p_training = p_samples.drop(p_validation.index).drop(p_testing.index)

# split the negative class samples to (70%, 10%, 20%) for (training, validation, testing) respectively 
n_testing = n_samples.sample(frac = 0.2)
n_validation = n_samples.drop(n_testing.index).sample(frac = 0.125)
n_training = n_samples.drop(n_validation.index).drop(n_testing.index)

# concatenating the 70% of p-class and n-class to form the training set
training_set = pd.concat([p_training, n_training], axis=0, ignore_index=True)

# concatenating the 10% of p-class and n-class to form the validation set
validation_set = pd.concat([p_validation, n_validation], axis=0, ignore_index=True)

# concatenating the 20% of p-class and n-class to form the testing set
testing_set = pd.concat([p_testing, n_testing], axis=0, ignore_index=True)

# display(training_set)
# display(validation_set)
# display(testing_set)


training_data = training_set.iloc[:,:-1]
training_class = training_set.iloc[:,-1]
# display(training_data)
# display(training_class)

validation_data = validation_set.iloc[:,:-1]
validation_class = validation_set.iloc[:,-1]
# display(testing_data)
# display(testing_class)

testing_data = testing_set.iloc[:,:-1]
testing_class = testing_set.iloc[:,-1]
# display(testing_data)
# display(testing_class)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
...,...,...
49983,"I loved it, having been a fan of the original ...",positive
49985,Imaginary Heroes is clearly the best film of t...,positive
49989,I got this one a few weeks ago and love it! It...,positive
49992,John Garfield plays a Marine who is blinded by...,positive


,review,sentiment
3,Basically there's a family where a little boy ...,negative
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
10,Phil the Alien is one of those quirky films wh...,negative
11,I saw this movie when I was about 12 when it c...,negative
...,...,...
49994,This is your typical junk comedy.<br /><br />T...,negative
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


,review,sentiment
0,A wonderful little production. <br /><br />The...,positive
1,I sure would like to see a resurrection of a u...,positive
2,If you like original gut wrenching laughter yo...,positive
3,"I remember this film,it was the first film i h...",positive
4,After the success of Die Hard and it's sequels...,positive
...,...,...
34995,This is your typical junk comedy.<br /><br />T...,negative
34996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
34997,I am a Catholic taught in parochial elementary...,negative
34998,I'm going to have to disagree with the previou...,negative


,review,sentiment
0,"""Hell To Pay"" bills itself as the rebirth of t...",positive
1,"I know, I know: it's childish. But I just love...",positive
2,The first installment of this notorious horror...,positive
3,Absolutely fantastic! Whatever I say wouldn't ...,positive
4,It's hard to know what to make of this weird l...,positive
...,...,...
4995,Poor Whoopi Goldberg. Imagine her at a friend'...,negative
4996,I have not yet seen anyone slate this film and...,negative
4997,"""A lot of the films I've made probably could h...",negative
4998,"Last night, I am sitting in my TV room, beer i...",negative


,review,sentiment
0,What's the point of this messages if not to di...,positive
1,Enchanting. The best time to see this movie is...,positive
2,Tim Krabbe is the praised author of 'Het Goude...,positive
3,A noted cinematic phenomenon of the late eight...,positive
4,Patty Chayevsky was years ahead of most succes...,positive
...,...,...
9995,How many times do we have to see bad horror mo...,negative
9996,Extremely slow movie.There are fine performanc...,negative
9997,I won't be too hard on this show because I enj...,negative
9998,I watched this on a weekend afternoon as there...,negative


...

0        positive
1        positive
2        positive
3        positive
4        positive
           ...   
34995    negative
34996    negative
34997    negative
34998    negative
34999    negative
Name: sentiment, Length: 35000, dtype: object

,review
0,"I was born in Beijing, China and moved to the ..."
1,This is one of the most amazing stories I have...
2,I was adopted at birth and certainly did NOT h...
3,Nigel Balchin's maze-like novel 'A Way Through...
4,Who doesn't love the muppets?! Impossible it i...
...,...
9995,I know if I was a low budget film maker I woul...
9996,"I can't knock this film too terribly, because ..."
9997,Greg Davis and Bryan Daly take some crazed sta...
9998,Ha. without a doubt Tommy's the evil one here....


0       positive
1       positive
2       positive
3       positive
4       positive
          ...   
9995    negative
9996    negative
9997    negative
9998    negative
9999    negative
Name: sentiment, Length: 10000, dtype: object

,review
0,What's the point of this messages if not to di...
1,Enchanting. The best time to see this movie is...
2,Tim Krabbe is the praised author of 'Het Goude...
3,A noted cinematic phenomenon of the late eight...
4,Patty Chayevsky was years ahead of most succes...
...,...
9995,How many times do we have to see bad horror mo...
9996,Extremely slow movie.There are fine performanc...
9997,I won't be too hard on this show because I enj...
9998,I watched this on a weekend afternoon as there...


0       positive
1       positive
2       positive
3       positive
4       positive
          ...   
9995    negative
9996    negative
9997    negative
9998    negative
9999    negative
Name: sentiment, Length: 10000, dtype: object

# Text Pre-processing

# Classification using BERT